In [ ]:
pip install torch

In [2]:
import torch
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F 
import numpy as np
import random

In [3]:
# Define the network architecture 
class QNetwork(nn.Module):
def init (self, state_size, action_size): 
    super(QNetwork, self). init () self.fc1 = 
    nn.Linear(state_size, 64) self.fc2 = 
    nn.Linear(64, 64)
self.fc3 = nn.Linear(64, action_size)

def forward(self, x):
x = torch.relu(self.fc1(x)) 
x = torch.relu(self.fc2(x)) 
x = self.fc3(x)
return x


IndentationError: expected an indented block (4250441579.py, line 3)

In [ ]:
# Define the replay buffer class ReplayBuffer:
def init (self, capacity): self.capacity = capacity self.buffer = [] self.index = 0

def push(self, state, action, reward, next_state, done): if len(self.buffer) < self.capacity:
self.buffer.append(None)
self.buffer[self.index] = (state, action, reward, next_state, done) self.index = (self.index + 1) % self.capacity

def sample(self, batch_size):
batch = np.random.choice(len(self.buffer), batch_size, replace=False) states, actions, rewards, next_states, dones = [], [], [], [], []
for i in batch:
state, action, reward, next_state, done = self.buffer[i] states.append(state)
actions.append(action) rewards.append(reward) next_states.append(next_state)
 
dones.append(done) 
return (torch.tensor(np.array(states)).float(), torch.tensor(np.array(actions)).long(), torch.tensor(np.array(rewards)).unsqueeze(1).float(),
        torch.tensor(np.array(next_states)).float(), torch.tensor(np.array(dones)).unsqueeze(1).int())

def len  (self):
return len(self.buffer)


In [ ]:
# Define the Double DQN agent class DDQNAgent:
def init (self, state_size, action_size, seed, learning_rate=1e-3, capacity=1000000,
discount_factor=0.99, tau=1e-3, update_every=4, batch_size=64): self.state_size = state_size
self.action_size = action_size self.seed = seed self.learning_rate = learning_rate
self.discount_factor = discount_factor self.tau = tau
self.update_every = update_every self.batch_size = batch_size self.steps = 0

self.qnetwork_local = QNetwork(state_size, action_size) self.qnetwork_target = QNetwork(state_size, action_size) self.optimizer = optim.Adam(self.qnetwork_local.parameters(),
lr=learning_rate)
self.replay_buffer = ReplayBuffer(capacity) self.update_target_network()

def step(self, state, action, reward, next_state, done): # Save experience in replay buffer
self.replay_buffer.push(state, action, reward, next_state, done)

In [ ]:
# Learn every update_every steps self.steps += 1
if self.steps % self.update_every == 0:
if len(self.replay_buffer) > self.batch_size:
experiences = self.replay_buffer.sample(self.batch_size) self.learn(experiences)
 

def act(self, state, eps=0.0):
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") state = torch.from_numpy(state).float().unsqueeze(0).to(device) self.qnetwork_local.eval()
with torch.no_grad():
action_values = self.qnetwork_local(state) self.qnetwork_local.train()

# Epsilon-greedy action selection if random.random() > eps:
return np.argmax(action_values.cpu().data.numpy()) else:
return random.choice(np.arange(self.action_size))

def learn(self, experiences):
states, actions, rewards, next_states, dones = experiences

In [ ]:
# Get max predicted Q values (for next states) from target model Q_targets_next =
self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1) # Compute Q targets for current states
Q_targets = rewards + self.discount_factor * (Q_targets_next * (1 -
dones))

# Get expected Q values from local model
Q_expected = self.qnetwork_local(states).gather(1, actions.view(-1, 1))

# Compute loss
loss = F.mse_loss(Q_expected, Q_targets) # Minimize the loss self.optimizer.zero_grad() loss.backward()
self.optimizer.step()   # Update target network self.soft_update(self.qnetwork_local, self.qnetwork_target)

In [ ]:
def update_target_network(self):
# Update target network parameters with polyak averaging
for target_param, local_param in zip(self.qnetwork_target.parameters(), self.qnetwork_local.parameters()):
target_param.data.copy_(self.tau * local_param.data + (1.0 - self.tau) * target_param.data)